# Dual Numbers
Dual number of a real number $a$ is of the form $a+b\epsilon$ where $\epsilon$ is a very small number such that $\epsilon^2=0$, meaning that it is a very small number & $b$ is usually 1.

## Basic Arithmetic
### Addition / Subtraction
Adding & subtracting two dual numbers is similar to adding or subtracting two complex numbers, we deal with real and dual numbers seperately.
$$(3 + 4\epsilon) + (1 + 2\epsilon) = 4 + 6\epsilon$$
$$(3 + 4\epsilon) - (1 + 2\epsilon) = 2 + 2\epsilon$$

### Multiplication
For multiplying two dual numbers, we use FOIL(Firsts, Outers, Inners, Lasts) techniques.
$$(3+4\epsilon)*(1+2\epsilon)$$
$$3+8\epsilon^2+4\epsilon+6\epsilon$$
$$3+10\epsilon+8\epsilon^2$$
since $\epsilon^2=0$
$$3+10\epsilon$$

Generalizing, $(a+b\epsilon)*(c+d\epsilon)$,results in - 
$$a*c+(b*c+a*d)\epsilon$$

### Division
Dividing one dual number by other involves taking the conjugate of the divisor.
$$(3+4\epsilon)\div(1+2\epsilon)$$
$$(3+4\epsilon)*(1-2\epsilon) \div (1+2\epsilon)*(1-2\epsilon)$$
$$(3+4\epsilon-6*\epsilon-8\epsilon^2) \div (1-4\epsilon^2)$$
since $\epsilon^2=0$
$$(3-2\epsilon)\div1$$
Seperating the numerator
$$3-(2\epsilon)$$


Essentially, if numerator is $(a+b\epsilon)$ and denominator is $(c+d\epsilon)$, then division results in - 
$$a/c+[(b*c-a*d)/c^2]\epsilon$$

# Automatic Differentation

Automatic Differentiation allows you to calculate the derivative of a function while calculating the value of the function. Essentially, we get both the value $f(x)$ and $f'(x)$

While performing any binary operation between a real number and a dual number, we convert real number to it's dual form using 0 for the dual component. For example; calculating $3x$ at $x=4+1\epsilon$

Convert $3$ to it's dual form: $3+0\epsilon$.

$$(3+0\epsilon)*(4+1\epsilon)$$
$$12+0\epsilon+3\epsilon+0\epsilon^2$$
$$12+3\epsilon$$


Instead of calculating $f(a)$ if we calculate the function's value for it's dual number i.e. $f(a+b\epsilon)$, we end up getting $f(a)$ and $f'(a)$ both. For example, we will start with $f(x)=3x+2$ and calculate $f(4)$ and $f'(4)$.

* **Step1** - We convert our $4$ into a dual number, using $1$ for the dual component. It gives us $4+1\epsilon$
* **Step2** - Similarily, we convert the constant $3$ in $3x+4$ into a dual number, using $0$ for the dual component.
* **Step3** - Perform multiplication as did in previous section
$$(3+0\epsilon)*(4+1\epsilon)$$
$$12+0\epsilon+3\epsilon+0\epsilon^2$$
$$12+3\epsilon$$
* **Step4** - Lastly, we need to add the constant 2, using 0 again for the dual component since it’s just a constant.
$$(12+3\epsilon)+(2+0\epsilon)$$
$$14+3\epsilon$$
* **Step5** - In the result above the real component $14$ gives us $f(x=4)$ and the dual component gives $f'(x=4)$

This way we can calculate the value as well as the derivative of any function at any value by calculating it's value for the corresponding dual number.

## Observations

1. We have a completely mechanical way to evaluate derivatives at a point. **This mechanism works for any function that can be composed out of the primitive operations that we have extended to dual numbers.** 
2. It turns every arithmetic operation into a fixed number of new arithmetic operations, so the **arithmetic cost goes up by a small constant factor.** For example, addition requires 2 extra steps. Similarily, multiplication requires 3 additional steps and so on. 
3. We also do not introduce any gross numerical sins, so we can reasonably expect the accuracy of the derivative to be no worse than that of the primal computation.3

## Implementation

In [2]:
from scipy.misc import derivative

In [10]:
class DualNumber:
    def __init__(self, real, dual):
        self.real = real
        self.dual = dual

    def __add__(self, other):
        if isinstance(other, DualNumber):
            return DualNumber(self.real + other.real, 
                              self.dual + other.dual)
        else:
            return DualNumber(self.real + other, self.dual)
    # addition is commutative
    __radd__ = __add__

    def __sub__(self, other):
        if isinstance(other, DualNumber):
            return DualNumber(self.real - other.real,
                              self.dual - other.dual)
        else:
            return DualNumber(self.real - other, self.dual)

    def __rsub__(self, other):
        return DualNumber(other, 0) - self

    def __mul__(self, other):
        if isinstance(other, DualNumber):
            return DualNumber(self.real * other.real,
                              self.real * other.dual + self.dual * other.real)
        else:
            return DualNumber(self.real * other, self.dual * other)
    # multiplication is commutative
    __rmul__ = __mul__

    def __div__(self, other):
        if isinstance(other, DualNumber):
            return DualNumber(self.real/other.real,
                              (self.dual*other.real - self.real*other.dual)/(other.real**2))
        else:
            return (1/other) * self

    def __rtruediv__(self, other):
        return DualNumber(other, 0).__div__(self)

    def __pow__(self, other):
        return DualNumber(self.real**other,
                          self.dual * other * self.real**(other - 1))

    def __repr__(self):
        return repr(self.real) + ' + ' + repr(self.dual) + '*epsilon'

In [11]:
def auto_diff(f, x):
    return f(DualNumber(x, 1.)).dual

$$f(x)=1/x^5$$

In [27]:
x = 0.1
f = lambda x: 1./x**5
print(f"[AUTODIFF] f'({x}): {auto_diff(f, x):.4f}") 
print(f"[SCIPY] f'({x}): {derivative(f, x0=x, n=1, dx=1e-6):.4f}")

[AUTODIFF] f'(0.1): -5000000.0000
[SCIPY] f'(0.1): -5000000.0035


$$f(x)=5x^2+4x+1$$

In [26]:
x = 2
f = lambda x: 5*(x**2)+4*x+1
print(f"[AUTODIFF] f'({x}): {auto_diff(f, x):.4f}") 
print(f"[SCIPY] f'({x}): {derivative(f, x0=x, n=1, dx=1e-6):.4f}")

[AUTODIFF] f'(2): 24.0000
[SCIPY] f'(2): 24.0000


## Further Work
It's clear that since the other numbers(5,3) are just constant in say $5x+3$, we consider 0 for their *dual component*. But, what is the thing about choosing $1$ for *dual components* while calculating $f(x)$ as $f(x+1\epsilon)$? 

$$\left. \frac{df(x)}{dx}\right|_x = \textrm{epsilon-coefficient}(\textrm{dual-version}(f)(x+1\epsilon)).$$

As far as I could think of, automatic differentiation worked becuase all of the above functions were composed of basic primary operations. How would things work if something like non-linear functions or arbitrary functions jump in? I mean, Taylor series gives a good approximation of $sin(x), cos(x)$ or any function at any point for that matter, so will we have to integrate them as some sort of class method in the above class? Unclear about it at this point.


How to extend this to say calculating second or third derivatives? Does automatic differentation hold for higher order derivatives? If yes, then what changes? For example, I read somewhere that for calculating second-order derivative, we have to make sure that $\epsilon^3=0$ instead of $\epsilon^2$